In [ ]:
!pip install -U \
  "langchain==0.3.27" \
  "langchain-core==0.3.74" \
  "langchain-community==0.3.27" \
  "langchain-openai==0.3.31" \
  "langgraph==0.2.45" \
  "openai==1.100.0" \
  "faiss-cpu==1.12.0" \
  "tiktoken==0.11.0" \
  "pypdf==5.1.0" \
  "tqdm==4.67.1" \
  "python-dotenv==1.0.1" \
  "pydantic==2.11.7" \
  "numpy==2.0.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 7.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.5/786.5 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 코랩 환경에서 실행 예시
# ========================

# 1. 환경 변수 설정
import os
os.environ["OPENAI_API_KEY"] = "fill out your API key"

In [ ]:
# =============================================================================
# 📌 LangGraph RAG (병렬: 법률 · 사례) — Router → [Law || Case] → Combine → Summarize → END
# - 병렬 분기: router가 law, case를 동시에 트리거
# - 델타 반환: 각 노드는 자신이 책임지는 키만 갱신(InvalidUpdateError 회피)
# - 조인/아이도포텐트: combine은 두 분기 완료 여부 확인 후 1회만 결과 생성
# =============================================================================
from __future__ import annotations
import os, re
from typing import TypedDict, List, Annotated, Dict, Any

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver

# -----------------------------------------------------------------------------
# 환경설정
# -----------------------------------------------------------------------------
if not os.environ.get("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY를 먼저 설정하세요. 예) os.environ['OPENAI_API_KEY']='sk-...'")

LLM = ChatOpenAI(model="gpt-4o-mini", temperature=0)
EMB = OpenAIEmbeddings(model="text-embedding-3-large")

# -----------------------------------------------------------------------------
# 벡터DB 경로
# -----------------------------------------------------------------------------
DB_LAW  = "fill out DB path"   
DB_CASE = "fill out DB path"  

VL = FAISS.load_local(DB_LAW,  EMB, allow_dangerous_deserialization=True)
VC = FAISS.load_local(DB_CASE, EMB, allow_dangerous_deserialization=True)

# -----------------------------------------------------------------------------
# 프롬프트
# -----------------------------------------------------------------------------
BASE_INSTRUCTION = (
    "당신은 개인정보 보호법에 정통한 법률 전문가입니다.\n"
    "제공된 문서만 근거로 답변하고, 불가능하면 'I don't know'로 응답하세요."
)

LAW_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        f"{BASE_INSTRUCTION}\n"
        "다음은 '개인정보 보호법/시행령/시행규칙' 발췌입니다:\n\n"
        "{context}\n\n"
        "질문: {question}\n\n"
        "[적용 법조문]\n- 법률명 제n조 제m항: 요지\n[분석]\n- 요건/의무/금지\n- 제재/쟁점\n"
        "※ 문서에 존재하는 조문만 정확히 표기."
    ),
)

CASE_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        f"{BASE_INSTRUCTION}\n"
        "다음은 '사례(분쟁조정/행정처분/판례 요약)' 발췌입니다:\n\n"
        "{context}\n\n"
        "질문: {question}\n\n"
        "첫 문장은 '유사 사례에 따르면'으로 시작하고,\n"
        "(1) 사실관계 (2) 쟁점 (3) 판단/결과 (4) 실무 포인트를 간결히 정리."
    ),
)

# -----------------------------------------------------------------------------
# 상태
# -----------------------------------------------------------------------------
class QAState(TypedDict):
    # 입력
    query: Annotated[str, "query"]
    # 라우팅 플래그
    need_law: bool
    need_case: bool
    # 완료 플래그
    done_law: bool
    done_case: bool
    combined_done: bool
    # 산출물
    result_law: str
    result_case: str
    law_citations: List[str]
    combined: str
    summary: str
    # 출처
    source_documents_law: List
    source_documents_case: List
    source_documents: List

# -----------------------------------------------------------------------------
# 리트리버/체인
# -----------------------------------------------------------------------------
ret_lw = VL.as_retriever(search_type="similarity", search_kwargs={"k": 2, "fetch_k": 5})
ret_cs = VC.as_retriever(search_type="similarity", search_kwargs={"k": 2, "fetch_k": 5})

chain_lw = RetrievalQA.from_chain_type(
    llm=LLM, retriever=ret_lw, return_source_documents=True,
    chain_type="stuff", chain_type_kwargs={"prompt": LAW_PROMPT},
)
chain_cs = RetrievalQA.from_chain_type(
    llm=LLM, retriever=ret_cs, return_source_documents=True,
    chain_type="stuff", chain_type_kwargs={"prompt": CASE_PROMPT},
)

# -----------------------------------------------------------------------------
# 조문 자동 추출
# -----------------------------------------------------------------------------
COMBINED_ARTICLE_RE = re.compile(r"(제\s*\d+\s*조(?:\s*의\s*\d+)?)\s*(?:[,·]?\s*제\s*(\d+)\s*항)?", re.UNICODE)

def extract_statute_citations(source_docs: List) -> List[str]:
    found, seen = [], set()
    for d in source_docs or []:
        text = getattr(d, "page_content", "") or ""
        meta = getattr(d, "metadata", {}) or {}
        law_name = (meta.get("law_name") or "").strip()
        for m in COMBINED_ARTICLE_RE.finditer(text):
            article = (m.group(1) or "").strip()
            clause  = (m.group(2) or "").strip()
            clause_str = f" 제{clause}항" if clause else ""
            prefix = f"{law_name} " if law_name else ""
            citation = f"{prefix}{article}{clause_str}".replace("  ", " ")
            if citation and citation not in seen:
                seen.add(citation); found.append(citation)
                if len(found) >= 8: return found
    return found

def _looks_unknown(text: str) -> bool:
    t = (text or "").lower()
    return len(t.strip()) < 20 or "i don't know" in t or "모르" in t

# -----------------------------------------------------------------------------
# 라우팅 규칙
# -----------------------------------------------------------------------------
OV_BOTH = ["[both]", "[종합]", "[all]"]
OV_LAW  = ["[law]", "[법]", "[조문]"]
OV_CASE = ["[case]", "[사례]"]
NEG_LAW  = ["[nolaw]", "[law_off]", "법은 빼", "조문 제외", "법률 제외"]
NEG_CASE = ["[nocase]", "[case_off]", "사례는 빼", "분쟁례 제외"]

def classify_needs(q: str) -> Dict[str, bool]:
    q = (q or "").strip()
    need_law, need_case = True, True
    if any(t in q for t in OV_BOTH): pass
    elif any(t in q for t in OV_LAW):  need_law, need_case = True, False
    elif any(t in q for t in OV_CASE): need_law, need_case = False, True
    if any(t in q for t in NEG_LAW):  need_law  = False
    if any(t in q for t in NEG_CASE): need_case = False
    return {"need_law": need_law, "need_case": need_case}

# -----------------------------------------------------------------------------
# 노드들 — 모두 "델타만" 반환
# -----------------------------------------------------------------------------
def router_node(state: QAState) -> Dict[str, Any]:
    needs = classify_needs(state["query"])
    return {"need_law": needs["need_law"], "need_case": needs["need_case"]}

def law_node(state: QAState) -> Dict[str, Any]:
    if not state.get("need_law", True):
        return {"result_law": "", "source_documents_law": [], "law_citations": [], "done_law": True}
    r = chain_lw.invoke({"query": state["query"]})
    cites = extract_statute_citations(r.get("source_documents", []))
    return {
        "result_law": r.get("result", "") or "",
        "source_documents_law": r.get("source_documents", []) or [],
        "law_citations": cites,
        "done_law": True,
    }

def case_node(state: QAState) -> Dict[str, Any]:
    if not state.get("need_case", True):
        return {"result_case": "", "source_documents_case": [], "done_case": True}
    r = chain_cs.invoke({"query": state["query"]})
    docs = r.get("source_documents", []) or []
    ans  = r.get("result", "") or ""
    if len(docs) == 0 or _looks_unknown(ans):
        return {"result_case": "", "source_documents_case": [], "done_case": True}
    return {"result_case": ans, "source_documents_case": docs, "done_case": True}

def combine_node(state: QAState) -> Dict[str, Any]:
    # 이미 한 번 생성했으면 통과
    if state.get("combined_done", False):
        return {}
    # 두 분기 완료 여부
    need_law  = state.get("need_law", True)
    need_case = state.get("need_case", True)
    ok_law  = (not need_law)  or state.get("done_law", False)
    ok_case = (not need_case) or state.get("done_case", False)
    if not (ok_law and ok_case):
        return {}  # 아직 대기

    parts = []
    if state.get("result_law"):  parts.append(state["result_law"])
    if state.get("result_case"): parts.append(state["result_case"])
    cites = state.get("law_citations") or []
    cites_line = ", ".join(cites)

    if not parts:
        combined_text = "I don't know"
    else:
        material = "\n\n".join(parts)
        prompt = (
            "아래 자료만 근거로, 개인정보보호 실무 답변을 자연스러운 한국어 문단으로 통합하라. "
            "라벨 금지, 최대 두 문단, 추정 금지.\n"
            f"{('참고 조문: ' + cites_line) if cites_line else ''}\n\n"
            "자료:\n" + material
        )
        combined_text = LLM.invoke(prompt).content

    all_sources = []
    all_sources += state.get("source_documents_law", [])
    all_sources += state.get("source_documents_case", [])
    return {"combined": combined_text, "source_documents": all_sources, "combined_done": True}

def summarize_node(state: QAState) -> Dict[str, Any]:
    text = state.get("combined", "") or ""
    if not text.strip():
        return {"summary": "I don't know"}
    cite_str = ", ".join(state.get("law_citations") or [])
    prompt = (
        "아래 통합 초안만을 근거로, 질문 의도에 맞춘 한 문단 의견서를 작성하라. "
        "라벨·불릿 금지, 5~10문장, 없는 정보 언급 금지, 결론은 문단 후반부.\n"
        f"{('참고 가능한 조문: ' + cite_str) if cite_str else ''}\n\n"
        "통합 초안:\n" + text
    )
    return {"summary": LLM.invoke(prompt).content}

# -----------------------------------------------------------------------------
# 그래프 구성 — 병렬 분기 + 단일 조인
# -----------------------------------------------------------------------------
def build_graph() -> Any:
    g = StateGraph(QAState)
    g.add_node("router",  router_node)
    g.add_node("law",     law_node)
    g.add_node("case",    case_node)
    g.add_node("combine", combine_node)
    g.add_node("summarize", summarize_node)

    g.set_entry_point("router")

    # 병렬 실행: router → law, case
    g.add_edge("router", "law")
    g.add_edge("router", "case")

    # 조인: 두 분기에서 모두 combine으로 흘러오되, combine이 스스로 준비 여부 점검
    g.add_edge("law",  "combine")
    g.add_edge("case", "combine")

    # 이후 요약 → END
    g.add_edge("combine", "summarize")
    g.add_edge("summarize", END)

    return g.compile(checkpointer=InMemorySaver())

# -----------------------------------------------------------------------------
# 실행 예시
# -----------------------------------------------------------------------------
def run_query(graph, question: str, thread_id: str = "privacy-rag-parallel") -> Dict[str, Any]:
    init: QAState = {
        "query": question,
        "need_law": True, "need_case": True,
        "done_law": False, "done_case": False,
        "combined_done": False,
        "result_law": "", "result_case": "",
        "law_citations": [], "combined": "", "summary": "",
        "source_documents_law": [], "source_documents_case": [], "source_documents": [],
    }
    return graph.invoke(init, config={"configurable": {"thread_id": thread_id}})

In [ ]:
# 그래프 빌드
graph = build_graph()

# 초기 상태
init_state = {
    "query": "병원이 환자의 진료기록을 제3자에게 동의 없이 제공한 경우 어떤 법적 문제가 있나요?",
    "need_law": True,
    "need_case": True,
    "result_law": "",
    "result_case": "",
    "law_citations": [],
    "combined": "",
    "summary": "",
    "source_documents_law": [],
    "source_documents_case": [],
    "source_documents": [],
}

# 실행
final_state = graph.invoke(init_state, config={"configurable": {"thread_id": "demo-run-1"}})

print("📌 질문:", final_state["query"])
print("\n📑 최종 요약 결과:\n", final_state["summary"])
print("\n📖 참고 조문:", final_state["law_citations"])


📌 질문: 병원이 환자의 진료기록을 제3자에게 동의 없이 제공한 경우 어떤 법적 문제가 있나요?

📑 최종 요약 결과:
 개인정보 보호법 제18조에 명시된 바와 같이, 개인정보처리자는 수집한 개인정보를 그 목적을 초과하여 이용하거나 제3자에게 제공할 수 없습니다. 특히 병원이 환자의 진료기록을 제3자에게 제공하고자 할 경우, 반드시 환자의 동의를 받아야 하며, 동의 없이 제공하는 것은 법적으로 금지됩니다. 이러한 규정은 정보주체의 권리를 보호하기 위한 중요한 장치로 작용합니다. 만약 병원이 동의 없이 진료기록을 제공한다면, 이는 개인정보 보호법 위반으로 간주되어 법적 책임을 초래할 수 있습니다. 따라서 병원은 개인정보를 제3자에게 제공하기 전에 반드시 수집 목적과의 관련성을 검토하고, 정보주체의 이익이 침해되지 않도록 주의해야 합니다. 결론적으로, 법적 문제를 예방하고 정보주체의 권리를 보호하기 위해서는 동의 절차와 안전성 확보 조치를 철저히 준수하는 것이 필수적입니다.

📖 참고 조문: ['제18조', '제15조 제1항', '제17조 제1항', '제28조의8 제1항', '제16조 제1항', '제16조 제3항', '제35조의2 제1항', '제38조 제1항']
